In [1]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

# Prepare model

In [ ]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

# Analyze execution time

In [34]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=20))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         1.97%       6.585ms       100.00%     334.010ms     334.010ms             1                                                                                []  
                     aten::conv2d         0.01%      49.401us        20.30%      67.796ms      16.949ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], 

# Analyze memory consumption

In [14]:
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))
# print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.22%     634.802us         0.22%     634.802us       3.174us      94.86 Mb      94.86 Mb           200  
    aten::max_pool2d_with_indices         5.04%      14.575ms         5.04%      14.575ms      14.575ms      11.48 Mb      11.48 Mb             1  
                      aten::addmm         0.10%     302.710us         0.11%     313.781us     313.781us      19.53 Kb      19.53 Kb             1  
                       aten::mean         0.01%      16.561us         0.03%      82.726us      82.726us      10.

# Tracing

In [ ]:
# activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA, ProfilerActivity.XPU]
activities = [ProfilerActivity.CPU]
with profile(activities=activities) as prof:
    model(inputs)

# Trace.json can be uploaded to: chrome://tracing/
prof.export_chrome_trace("trace.json")

In [37]:
# sort_by_keywrod = "self_" + device + "_time_total"
sort_by_keyword = "cpu_time_total"

with profile(
    activities=activities,
    with_stack=True,
) as prof:
    model(inputs)

# Print aggregated stats
print(prof.key_averages(group_by_stack_n=5).table(sort_by=sort_by_keyword,row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.06%     157.169us        81.88%     222.327ms      11.116ms            20  
                aten::convolution         0.22%     603.283us        81.82%     222.170ms      11.109ms            20  
               aten::_convolution         0.12%     326.985us        81.60%     221.567ms      11.078ms            20  
         aten::mkldnn_convolution        81.17%     220.412ms        81.48%     221.240ms      11.062ms            20  
                 aten::batch_norm         0.04%     107.433us         8.78%      23.831ms       1.192ms            20  
     aten::_batch_norm_impl_index       

# Use scheduler for tracing

In [41]:
sort_by_keyword = "cpu_time_total"

def trace_handler(p):
    output = p.key_averages().table(sort_by=sort_by_keyword, row_limit=10)
    print(output)
    p.export_chrome_trace("/tmp/trace_" + str(p.step_num) + ".json")

with profile(
    activities=activities,
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2),
    on_trace_ready=trace_handler
) as p:
    for idx in range(4):
        model(inputs)
        p.step()

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    ProfilerStep*         2.91%      11.963ms       100.00%     411.323ms     205.662ms             2  
                     aten::conv2d         0.07%     290.875us        80.47%     331.002ms       8.275ms            40  
                aten::convolution         0.26%       1.086ms        80.40%     330.711ms       8.268ms            40  
               aten::_convolution         0.15%     601.228us        80.14%     329.626ms       8.241ms            40  
         aten::mkldnn_convolution        79.65%     327.617ms        79.99%     329.024ms       8.226ms            40  
                 aten::batch_norm       